In [27]:


# %%

import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

# %%
tf.random.set_seed(10)
BATCH_SIZE = 128
BUFFER_SIZE = 10000

# %%

df = pd.read_csv("../data/datefrom1st.csv")
df.index = df.datetime
df = df.drop(
    ["temperature", "difference", "Unnamed: 0", 'datetime', 'percipitation', 'air_pressure', 'sea_level_pressure',
     'wind_degree'], axis=1)


In [28]:
df.corr()

,result,wind_speed,humidity,solar_radiation,solar_intensity
result,1.000000,0.403046,-0.394002,0.948446,0.795518
wind_speed,0.403046,1.000000,-0.232243,0.411511,0.336826
humidity,-0.394002,-0.232243,1.000000,-0.363429,-0.395207
solar_radiation,0.948446,0.411511,-0.363429,1.000000,0.822481
solar_intensity,0.795518,0.336826,-0.395207,0.822481,1.000000


In [29]:
TRAIN_SPLIT = int(len(df.index) * 0.8)
scaler = MinMaxScaler()
values = scaler.fit_transform(df)

In [34]:
df

,result,wind_speed,humidity,solar_radiation,solar_intensity
datetime,,,,,
2019-07-01 00:00:00,0.0,1.193333,84.453333,0.0,0.0
2019-07-01 00:15:00,0.0,1.306667,84.853333,0.0,0.0
2019-07-01 00:30:00,0.0,0.646667,85.280000,0.0,0.0
2019-07-01 00:45:00,0.0,1.053333,85.560000,0.0,0.0
2019-07-01 01:00:00,0.0,0.973333,85.740000,0.0,0.0
...,...,...,...,...,...
2020-06-30 23:00:00,0.0,2.140000,83.573333,0.0,0.0
2020-06-30 23:15:00,0.0,1.773333,84.540000,0.0,0.0
2020-06-30 23:30:00,0.0,1.606667,84.626667,0.0,0.0


In [31]:
train = values[:TRAIN_SPLIT, :]
test = values[TRAIN_SPLIT:, :]
# split into input and outputs
train_X, train_y = train[:, 1:], train[:, 0]
test_X, test_y = test[:, 1:], test[:, 0]
# %%

train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)



(28109, 1, 4) (28109,) (7028, 1, 4) (7028,)


In [32]:
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()

    plt.show()


# %%

def show_plot(plot_data, delta, title):
    labels = ['History', 'True Future', 'Model Prediction']
    marker = ['.-', 'rx', 'go']
    time_steps = create_time_steps(plot_data[0].shape[0])
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, x in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10,
                     label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel('Time-Step')
    return plt


def create_time_steps(length):
    return list(range(-length, 0))


# %%

def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12, 6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)

    plt.plot(num_in, np.array(history[:, 1]), label='History')
    plt.plot(np.arange(num_out) / STEP, np.array(true_future), 'bo',
             label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out) / STEP, np.array(prediction), 'ro',
                 label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()
    plt.savefig("output.png")


# %%
import tensorflow as tf
from tensorflow.keras.layers import RepeatVector

mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    multi_step_model = tf.keras.models.Sequential()
    multi_step_model.add(tf.keras.layers.LSTM(100,activation="relu",
                                              return_sequences=True,
                                              input_shape=train_X.shape[1:]))
    multi_step_model.add(tf.keras.layers.Dense(1))

# %%
multi_step_model.summary()

# %%
from matplotlib import pyplot
EPOCHS = 20
EVALUATION_INTERVAL = 200
from tensorflow.keras.utils import multi_gpu_model

multi_step_model = multi_gpu_model(multi_step_model, gpus=4)

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

history = multi_step_model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
pyplot.savefig("test.png")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 100)            42000     
_________________________________________________________________
dense (Dense)                (None, 1, 1)              101       
Total params: 42,101
Trainable params: 42,101
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use `tf.distribute.MirroredStrategy` instead.


ValueError: To call `multi_gpu_model` with `gpus=4`, we expect the following devices to be available: ['/cpu:0', '/gpu:0', '/gpu:1', '/gpu:2', '/gpu:3']. However this machine only has: ['/cpu:0', '/xla_cpu:0']. Try reducing `gpus`.

In [33]:
from numpy import concatenate
from math import sqrt
from sklearn.metrics import mean_squared_error


# make a prediction
yhat = multi_step_model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [35]:
inv_y.shape[0]


NameError: name 'inv_y' is not defined